### **0. Library**

In [28]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

### **1. Data Load**

In [50]:
# 데이터 로드

train_path = '/root/dev/Ml_Team2_fork/1.Data/train.csv'
test_path  = '/root/dev/Ml_Team2_fork/1.Data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [51]:
# 문자열 변경을 위한 re import
import re

key_list = {}

for text in list(train.columns):
    # '전용면적별세대현황'으로 시작하면 괄호 제거 안함
    if text.startswith('k-전용면적별세대'): # 그냥 괄호를 제거하면 중복된 이름이 발생하기 때문에 처리한 예외 처리 
        cleaned = re.sub(r'^k-', '', text).strip()

    else :
        cleaned = re.sub(r'\(.*?\)','',text)
        cleaned = re.sub(r'^k-','',cleaned)
        cleaned = cleaned.strip()
    key_list[text] = cleaned
print(key_list)

train = train.rename(columns = key_list)
test = test.rename(columns = key_list)

{'시군구': '시군구', '번지': '번지', '본번': '본번', '부번': '부번', '아파트명': '아파트명', '전용면적(㎡)': '전용면적', '계약년월': '계약년월', '계약일': '계약일', '층': '층', '건축년도': '건축년도', '도로명': '도로명', '해제사유발생일': '해제사유발생일', '등기신청일자': '등기신청일자', '거래유형': '거래유형', '중개사소재지': '중개사소재지', 'k-단지분류(아파트,주상복합등등)': '단지분류', 'k-전화번호': '전화번호', 'k-팩스번호': '팩스번호', '단지소개기존clob': '단지소개기존clob', 'k-세대타입(분양형태)': '세대타입', 'k-관리방식': '관리방식', 'k-복도유형': '복도유형', 'k-난방방식': '난방방식', 'k-전체동수': '전체동수', 'k-전체세대수': '전체세대수', 'k-건설사(시공사)': '건설사', 'k-시행사': '시행사', 'k-사용검사일-사용승인일': '사용검사일-사용승인일', 'k-연면적': '연면적', 'k-주거전용면적': '주거전용면적', 'k-관리비부과면적': '관리비부과면적', 'k-전용면적별세대현황(60㎡이하)': '전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)': '전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하': '85㎡~135㎡이하', 'k-135㎡초과': '135㎡초과', 'k-홈페이지': '홈페이지', 'k-등록일자': '등록일자', 'k-수정일자': '수정일자', '고용보험관리번호': '고용보험관리번호', '경비비관리형태': '경비비관리형태', '세대전기계약방법': '세대전기계약방법', '청소비관리형태': '청소비관리형태', '건축면적': '건축면적', '주차대수': '주차대수', '기타/의무/임대/임의=1/2/3/4': '기타/의무/임대/임의=1/2/3/4', '단지승인일': '단지승인일', '사용허가여부': '사용허가여부', '관리비 업로드': '관리비 업로드

In [52]:
# Data pre-processing

## is_test : train/test 구분을 위한 변수
## concat을 통한 train, test 합쳐주기

train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])

print(concat.shape)
print(concat['is_test'].value_counts())

(1128094, 53)
0    1118822
1       9272
Name: is_test, dtype: int64


In [53]:
# 결측치는 아닌데 의미 없는 형식적 값 찾는 함수

def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if df[col].dtype == 'object':
            val_counts = df[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)
for col, vals in fake_nulls.items():
    print(f"🔎 {col} 컬럼에서 의미 없는 값 발견:")
    print(vals)
    print()

# 아무 의미 없는 값 결측처리를 위해 nan으로 변경
concat['도로명'] = concat['도로명'].replace(' ', np.nan)
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

🔎 도로명 컬럼에서 의미 없는 값 발견:
     1211
Name: 도로명, dtype: int64

🔎 등기신청일자 컬럼에서 의미 없는 값 발견:
     1111271
Name: 등기신청일자, dtype: int64

🔎 거래유형 컬럼에서 의미 없는 값 발견:
-    1086451
Name: 거래유형, dtype: int64

🔎 중개사소재지 컬럼에서 의미 없는 값 발견:
-    1090013
Name: 중개사소재지, dtype: int64

🔎 시행사 컬럼에서 의미 없는 값 발견:
.    868
-    152
Name: 시행사, dtype: int64

🔎 홈페이지 컬럼에서 의미 없는 값 발견:
없음    3628
.       72
Name: 홈페이지, dtype: int64



In [54]:
# 결측률 30% 이상 삭제
concat.drop(axis = 1, columns = list(concat.columns[concat.isnull().sum()/concat.shape[0] >= 0.3]), inplace=True)

In [55]:
# categorical column name, numerical column name return function
def split_categorical_numerical(df:pd.DataFrame, verbose:bool = True):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
    if verbose:
        print("📌 범주형 변수:", categorical_cols)
        print("📌 연속형 변수:", numerical_cols)
    return categorical_cols, numerical_cols

# update and check variable
cat_cols, num_cols = split_categorical_numerical(concat)

📌 범주형 변수: ['시군구', '번지', '아파트명', '도로명']
📌 연속형 변수: ['본번', '부번', '전용면적', '계약년월', '계약일', '층', '건축년도', 'target', 'is_test']


In [56]:
concat.drop(axis = 1, columns=['본번', '부번', '계약일', '번지', '아파트명', '도로명'])

,시군구,전용면적,계약년월,층,건축년도,target,is_test
0,서울특별시 강남구 개포동,79.97,201712,3,1987,124000.0,0
1,서울특별시 강남구 개포동,79.97,201712,4,1987,123500.0,0
2,서울특별시 강남구 개포동,54.98,201712,5,1987,91500.0,0
3,서울특별시 강남구 개포동,79.97,201801,4,1987,130000.0,0
4,서울특별시 강남구 개포동,79.97,201801,2,1987,117000.0,0
...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,84.65,202307,13,2014,NaN,1
9268,서울특별시 중랑구 신내동,84.62,202307,12,2014,NaN,1
9269,서울특별시 중랑구 신내동,101.65,202308,12,2014,NaN,1
9270,서울특별시 중랑구 신내동,84.94,202309,18,2014,NaN,1


In [57]:
concat['contract_month'] = concat['계약년월'] % 100 # 계약월
concat['contract_date'] = concat['계약년월']//100 + concat['contract_month'] / 12 # 계약시간

concat['covid'] = (concat['계약년월'] >= 202001).astype(int) # 코로나 시기 구분

# 빌딩의 나이
current = 2025
concat['apt_age'] = current - concat['건축년도']

# 
concat['구'] = list(map(lambda x : x.split(' ')[1],concat['시군구']))
concat['동'] = list(map(lambda x : x.split(' ')[2],concat['시군구']))

concat.drop(axis = 1, columns = ['건축년도', '계약년월', '시군구', '계약일', '본번', '부번', '도로명', '번지', '아파트명'], inplace = True)

In [58]:
cat_cols, num_cols = split_categorical_numerical(concat)

📌 범주형 변수: ['구', '동']
📌 연속형 변수: ['전용면적', '층', 'target', 'is_test', 'contract_month', 'contract_date', 'covid', 'apt_age']


In [59]:
# train set, test set split
df_train = concat.query('is_test == 0')
df_test = concat.query('is_test == 1')

# 이제 is_test 칼럼은 drop해줍니다.
df_train.drop(['is_test'], axis = 1, inplace=True)
df_test.drop(['is_test'], axis = 1, inplace=True)
print(df_train.shape, df_test.shape)

(1118822, 9) (9272, 9)


In [60]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( cat_cols ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( df_train[col].astype(str) )
    df_train[col] = lbl.transform(df_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(df_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    df_test[col] = lbl.transform(df_test[col].astype(str))

100%|██████████| 2/2 [00:00<00:00,  4.71it/s]


In [ ]:
# Target과 독립변수들을 분리해줍니다.
y_train = np.log1p(df_train['target'])
X_train = df_train.drop(['target'], axis=1)
df_test.drop(axis = 1, columns=['target'], inplace = True)

,전용면적,층,contract_month,contract_date,covid,apt_age,구,동
0,79.9700,5,7,2023.583333,1,38,0,8
1,108.2017,10,8,2023.666667,1,4,0,8
2,161.0000,15,7,2023.583333,1,41,0,8
3,133.4600,14,8,2023.666667,1,41,0,8
4,104.4300,6,8,2023.666667,1,41,0,8
...,...,...,...,...,...,...,...,...
9267,84.6500,13,7,2023.583333,1,11,24,174
9268,84.6200,12,7,2023.583333,1,11,24,174
9269,101.6500,12,8,2023.666667,1,11,24,174
9270,84.9400,18,9,2023.750000,1,11,24,174
